In [ ]:
# Log into Hugging Face's platform
from huggingface_hub import notebook_login
notebook_login()
# ---- End of your code ----


In [ ]:
%pip install gradio


In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

df = pd.read_csv('https://raw.githubusercontent.com/DiwaPrasetyo02/My-Machine-Learning-Project/refs/heads/main/dataset/Symptom2Disease.csv')
df.rename(columns={'label': 'Conference'}, inplace=True)
df.rename(columns={'text': 'Title'}, inplace=True)
df.head()

,Unnamed: 0,Conference,Title
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [ ]:
df['Conference'].value_counts()

,count
Conference,
Psoriasis,50
Varicose Veins,50
Typhoid,50
Chicken pox,50
Impetigo,50
Dengue,50
Fungal infection,50
Common Cold,50
Pneumonia,50


In [ ]:
possible_labels = df.Conference.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Psoriasis': 0,
 'Varicose Veins': 1,
 'Typhoid': 2,
 'Chicken pox': 3,
 'Impetigo': 4,
 'Dengue': 5,
 'Fungal infection': 6,
 'Common Cold': 7,
 'Pneumonia': 8,
 'Dimorphic Hemorrhoids': 9,
 'Arthritis': 10,
 'Acne': 11,
 'Bronchial Asthma': 12,
 'Hypertension': 13,
 'Migraine': 14,
 'Cervical spondylosis': 15,
 'Jaundice': 16,
 'Malaria': 17,
 'urinary tract infection': 18,
 'allergy': 19,
 'gastroesophageal reflux disease': 20,
 'drug reaction': 21,
 'peptic ulcer disease': 22,
 'diabetes': 23}

In [ ]:
df['label'] = df.Conference.replace(label_dict)

<ipython-input-6-51e75abc66fa>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df.Conference.replace(label_dict)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.2,
                                                  random_state=42,
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Conference', 'label', 'data_type']).count()

Unnamed: 0  Title
Conference                      label data_type                   
Acne                            11    train              40     40
                                      val                10     10
Arthritis                       10    train              40     40
                                      val                10     10
Bronchial Asthma                12    train              40     40
                                      val                10     10
Cervical spondylosis            15    train              40     40
                                      val                10     10
Chicken pox                     3     train              40     40
                                      val                10     10
Common Cold                     7     train              40     40
                                      val                10     10
Dengue                          5     train              40     40
                                      val                10     10
Dimorphic Hemorrhoids           9     train              40     40
                                      val                10     10
Fungal infection                6     train              40     40
                                      val                10     10
Hypertension                    13    train              40     40
                                      val                10     10
Impetigo                        4     train              40     40
                                      val                10     10
Jaundice                        16    train              40     40
                                      val                10     10
Malaria                         17    train              40     40
                                      val                10     10
Migraine                        14    train              40     40
                                      val                10     10
Pneumonia                       8     train              40     40
                                      val                10     10
Psoriasis                       0     train              40     40
                                      val                10     10
Typhoid                         2     train              40     40
                                      val                10     10
Varicose Veins                  1     train              40     40
                                      val                10     10
allergy                         19    train              40     40
                                      val                10     10
diabetes                        23    train              40     40
                                      val                10     10
drug reaction                   21    train              40     40
                                      val                10     10
gastroesophageal reflux disease 20    train              40     40
                                      val                10     10
peptic ulcer disease            22    train              40     40
                                      val                10     10
urinary tract infection         18    train              40     40
                                      val                10     10

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1',
                                          do_lower_case=True)

encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Title.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Title.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad

In [ ]:
model = BertForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

epochs = 15

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import os

# Membuat direktori 'data_volume' jika belum ada
if not os.path.exists('data_volume'):
    os.makedirs('data_volume')

# Setelah itu, lanjutkan dengan menyimpan model
torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epochs}.model')


In [ ]:
import random
import numpy as np
import torch
from tqdm.notebook import tqdm

from sklearn.metrics import f1_score
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}") # Print device information

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Move the model to the device
model.to(device) # This line is crucial

def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

for epoch in tqdm(range(1, epochs+1)):

    model.train()

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

Using device: cuda


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/320 [00:00<?, ?it/s]


Epoch 1
Training loss: 2.9588352996855973
Validation loss: 2.3645237550139426
F1 Score (Weighted): 0.4678960611962861


Epoch 2:   0%|          | 0/320 [00:00<?, ?it/s]

...


Epoch 13
Training loss: 0.01137742738937959
Validation loss: 0.05255476072779856
F1 Score (Weighted): 0.9916457811194653


Epoch 14:   0%|          | 0/320 [00:00<?, ?it/s]


Epoch 14
Training loss: 0.010326249204808846
Validation loss: 0.051843011868186295
F1 Score (Weighted): 0.9916457811194653


Epoch 15:   0%|          | 0/320 [00:00<?, ?it/s]


Epoch 15
Training loss: 0.009912034143053462
Validation loss: 0.0519048782531172
F1 Score (Weighted): 0.9916457811194653


In [ ]:
model = BertForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-15-240cc71e8109>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_gl

Class: Psoriasis
Accuracy: 1/10

Class: Varicose Veins
Accuracy: 0/10

Class: Typhoid
Accuracy: 0/10

Class: Chicken pox
Accuracy: 5/10

Class: Impetigo
Accuracy: 9/10

Class: Dengue
Accuracy: 1/10

Class: Fungal infection
Accuracy: 10/10

Class: Common Cold
Accuracy: 3/10

Class: Pneumonia
Accuracy: 10/10

Class: Dimorphic Hemorrhoids
Accuracy: 8/10

Class: Arthritis
Accuracy: 10/10

Class: Acne
Accuracy: 9/10

Class: Bronchial Asthma
Accuracy: 0/10

Class: Hypertension
Accuracy: 9/10

Class: Migraine
Accuracy: 10/10

Class: Cervical spondylosis
Accuracy: 10/10

Class: Jaundice
Accuracy: 10/10

Class: Malaria
Accuracy: 7/10

Class: urinary tract infection
Accuracy: 1/10

Class: allergy
Accuracy: 3/10

Class: gastroesophageal reflux disease
Accuracy: 10/10

Class: drug reaction
Accuracy: 0/10

Class: peptic ulcer disease
Accuracy: 3/10

Class: diabetes
Accuracy: 0/10



In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the tokenizer and the model
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')
model = BertForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

# Load the trained model state
model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))
model.eval()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-e253c71e0dc9>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://githu

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Ensure all tensors in the model are contiguous
for param in model.parameters():
    if not param.is_contiguous():
        param.data = param.contiguous()

# Push the model to the hub
model.push_to_hub(repo_id='my-finetuned-biobert')


model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/diwa02/my-finetuned-biobert/commit/7dde14e2fde50458c144af377b2eec473fcf430b', commit_message='Upload BertForSequenceClassification', commit_description='', oid='7dde14e2fde50458c144af377b2eec473fcf430b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/diwa02/my-finetuned-biobert', endpoint='https://huggingface.co', repo_type='model', repo_id='diwa02/my-finetuned-biobert'), pr_revision=None, pr_num=None)

In [ ]:
import torch
import torch.nn.functional as F
import gradio as gr

# Fungsi preprocessing (jika diperlukan)
def preprocess_text(text):
    # Misalnya, lakukan lowercasing dan hilangkan tanda baca
    text = text.lower()
    # Tambahkan preprocessing lainnya yang dilakukan saat pelatihan, seperti menghapus tanda baca jika perlu
    return text

# Definisikan fungsi predict_disease
def predict_disease(text):
    # Preprocessing teks sebelum diprediksi
    processed_text = preprocess_text(text)

    # Tokenize the input text
    inputs = tokenizer(processed_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Move inputs to the appropriate device
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Calculate probabilities using softmax
    probabilities = F.softmax(logits, dim=1)
    confidence, predicted_class = torch.max(probabilities, dim=1)
    confidence_percentage = confidence.item() * 100

    # Map the label to its corresponding disease name
    disease_name = list(label_dict.keys())[list(label_dict.values()).index(predicted_class.item())]

    return f"Penyakit: {disease_name}\nConfidence: {confidence_percentage:.2f}%"


In [ ]:
# Define the interface
interface = gr.Interface(
    fn=predict_disease,  # Function to make predictions
    inputs=gr.Textbox(lines=2, placeholder="Enter the description of symptoms here..."),  # Input box for user input
    outputs="text",  # Output type is text
    title="Disease Prediction using BioBERT",
    description="Enter the symptoms or description of the patient's condition, and the model will predict the most likely disease along with the confidence percentage.",
)

# Launch the interface
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f6a0ce5512a0a31de3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
